# Dog breed multi-class classifier

> Building a multi class image classifier with tensorflow.

## Problem

> Identifying a breed of a dog given an image of a dog.

## Data

> Can be found [here](https://www.kaggle.com/c/dog-breed-identification/data?select=sample_submission.csv).
